In [1]:
import cPickle
import os
import numpy as np
import nltk
import nltk.corpus
import pandas as pd
import sys
import multiprocessing as mp
import text_tok
import random
import re

from nltk import sent_tokenize
from ast import literal_eval
from datetime import datetime

# Set the number of cores to use
NUM_CORE = mp.cpu_count()-4

# Truecasing

A small fraction of the dpa articles were missing all uppercase letters. To fix this, we used a truecasing model by Nils Reimers
(https://github.com/nreimers/truecaser), which is based on the work of Lucian Vlad Lita et al. (2003).

In [2]:
# Set the path to the truecasing model
path = os.path.join(os.getcwd(), 'truecaser-master')
os.chdir(path)
sys.path.insert(1, os.getcwd().replace('\\truecaser-master', ''))

In [3]:
from TrainFunctions import *
from EvaluateTruecaser import evaluateTrueCaser

In [4]:
# Initialize frequency distributions for bigrams and trigrams
uniDist = nltk.FreqDist()
backwardBiDist = nltk.FreqDist() 
forwardBiDist = nltk.FreqDist() 
trigramDist = nltk.FreqDist() 
wordCasingLookup = {}

In [5]:
# Load training data
data = pd.read_csv(os.path.join(os.getcwd().replace('truecaser-master', ''), 'dpa_prepro_step11.csv'), encoding = 'utf-8', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

In [6]:
# Select the subset of data (1,000,000 articles) for training the truecasing model
data_sample = data['texts'][1000000:2000000]

During the preprocessing of the training set, texts are tokenized and umlauts are replaced with their respective non-umlaut equivalents using the function `text_tok`. This step is necessary as the dataset to which the model will be applied has umlauts already replaced.

In [7]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    tokens = pool.map(text_tok.text_tok, [text for text in data_sample]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:03:05.189000


In [8]:
startTime = datetime.now()

# Train the truecasing model using 1,000,000 articles from the dpa corpus, where the capitalization is correct
updateDistributionsFromSentences(tokens, wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

# Save the frequency distributions
f = open('distributions.obj', 'wb')
cPickle.dump(uniDist, f, -1)
cPickle.dump(backwardBiDist, f, -1)
cPickle.dump(forwardBiDist, f, -1)
cPickle.dump(trigramDist, f, -1)
cPickle.dump(wordCasingLookup, f, -1)
f.close()


print(datetime.now()-startTime)

1:23:22.734000


Evaluate the truecasing model using 10 manually chosen sentences.

In [9]:
testSentences = [u"Obwohl der Erwerb von Wohnungen seit 1974 f\xfcr Ausl\xe4nder weitgehend verboten ist, h\xe4tten diese in zahlreichen F\xe4llen \xfcber \xf6sterreichische Strohm\xe4nner Wohnungen gekauft."
                ,u"So seien \xabMenschen im \xf6sterreichischen Altersheim\xbb als K\xe4ufer aufgetreten, die ihr \xabEigentum\xbb dem deutschen Anleger \xabvererbt\xbb h\xe4tten."
                ,u"Die Zusammensetzung der k\xfcnftigen SPD-Fraktionsspitze um den neuen Vorsitzenden Hans-Ulrich Klose nimmt allm\xe4hlich Konturen an."
                ,u"Die bei der Wahl des Vorsitzenden unterlegenen Mitbewerber Herta D\xe4ubler-Gmelin und Rudolf Dre\xdfler sollen dem Vernehmen nach zum k\xfcnftigen Team um Klose geh\xf6ren."
                ,u"Dieser will seine Pl\xe4ne f\xfcr eine gestraffte Fraktionsf\xfchrung am 10. Dezember den sozialdemokratischen Abgeordneten zur Abstimmung vorlegen."
                ,u"Sie sollen nicht mehr wie bisher gleichzeitig als Arbeitskreisleiter f\xfcr bestimmte Fachgebiete zust\xe4ndig sein, sondern zu allen Themen \xabvom Paragraphen 218 bis zur Abschaffung der Gewerbekapitalsteuer\xbb kompetent Stellung nehmen k\xf6nnen."
                ,u"Es sei denkbar, da\xdf es auch Gegenkandidaten zu den Vorschl\xe4gen Kloses geben wird."
                ,u"D\xe4ubler-Gmelin und Dre\xdfler w\xfcrden \xabmit Sicherheit eine herausgehobene Rolle haben, wenn sie es wollen\xbb, sagte Struck."
                ,u"Parteichef Bj\xf6rn Engholm sagte vor dem Parteirat zu den Verlierern der Wahl vor zehn Tagen: \xabIch w\xfcnsche mir, da\xdf Ihr beide und Hans-Ulrich Klose ein enges Team bildet, um zu zeigen: Sieger und Unterlegene ziehen an einem Strang."
                ,u"Bei der Leitung der Arbeitskreise, deren Zahl ebenfalls deutlich verringert werden soll, k\xf6nnten \xabneue K\xf6pfe\xbb in die F\xfchrung kommen. Zur Disposition st\xfcnden grunds\xe4tzlich folgende Politiker und Arbeitsgebiete: D\xe4ubler-Gmelin (Recht), Dre\xdfler (Soziales), Matth\xe4us-Maier (Finanzen), Norbert Gansel (Ausw\xe4rtiges), Willfried Penner (Inneres) Ingrid Becker-Inglau (Frauen), Harald Sch\xe4fer (Umwelt), Wolfgang Roth (Wirtschaft) und Thierse (ohne Arbeitskreis)."
                ]

In [10]:
def replace_umlauts(text):
        """This function replaces German umlauts with their respective substitutes."""
        replacements = {
            u'ä': u'ae',
            u'ö': u'oe',
            u'ü': u'ue',
            u'Ä': u'AE',
            u'Ö': u'OE',
            u'Ü': u'UE',
            u'ß': u'ss'
        }
        for umlaut, substitute in replacements.items():
            text = text.replace(umlaut, substitute)
        return text

In [11]:
# Apply the replace_umlauts function to the test sentences
testSentences = [replace_umlauts(sent) for sent in testSentences]

In [12]:
evaluateTrueCaser(testSentences, wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

[u'Parteichef', u'Bjoern', u'Engholm', u'sagte', u'vor', u'dem', u'Parteirat', u'zu', u'den', u'Verlierern', u'der', u'Wahl', u'vor', u'zehn', u'Tagen', u':', u'\xab', u'Ich', u'wuensche', u'mir', u',', u'dass', u'Ihr', u'beide', u'und', u'Hans-Ulrich', u'Klose', u'ein', u'enges', u'Team', u'bildet', u',', u'um', u'zu', u'zeigen', u':', u'Sieger', u'und', u'Unterlegene', u'ziehen', u'an', u'einem', u'Strang', u'.']
[u'Parteichef', u'Bjoern', u'Engholm', u'sagte', u'vor', u'dem', u'Parteirat', u'zu', u'den', u'Verlierern', u'der', u'Wahl', u'vor', u'zehn', u'Tagen', u':', u'\xab', u'Ich', u'wuensche', u'mir', u',', u'dass', u'ihr', u'beide', u'und', u'Hans-Ulrich', u'Klose', u'ein', u'enges', u'Team', u'bildet', u',', u'um', u'zu', u'zeigen', u':', u'Sieger', u'und', u'unterlegene', u'ziehen', u'an', u'einem', u'Strang', u'.']
-------------------
Accuracy: 99.34%


Evaluate the truecasing model using 100 randomly chosen sentences from 1000 articles that were not utilized in the model training process.

In [13]:
data_test = data['texts'][2000000:2001000]

data_test = data_test.tolist()
sentences = []

# Split the texts into sentences
for text in data_test:
    sentences.extend(sent_tokenize(text))
    
# Pick 100 random sentences
random_sentences = random.sample(sentences, 100)

In [14]:
# Apply the replace_umlauts function to the test sentences
random_sentences = [replace_umlauts(sent) for sent in random_sentences]

In [15]:
evaluateTrueCaser(random_sentences, wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

[u'Rund', u'die', u'Haelfte', u'der', u'Stellen', u'faellt', u'weg', u'in', u'der', u'Flaeche', u',', u'wo', u'unter', u'anderem', u'die', u'Vertriebsstruktur', u'verschlankt', u'wird', u'und', u'auch', u'Fuehrungspositionen', u'gestrichen', u'werden', u'.']
[u'Rund', u'die', u'Haelfte', u'der', u'Stellen', u'faellt', u'Weg', u'in', u'der', u'Flaeche', u',', u'wo', u'unter', u'anderem', u'die', u'Vertriebsstruktur', u'verschlankt', u'wird', u'und', u'auch', u'Fuehrungspositionen', u'gestrichen', u'werden', u'.']
-------------------
[u'Zudem', u'verbringen', u'sie', u'weniger', u'Zeit', u'zu', u'Hause', u',', u'weil', u'feste', u'Fernsehzeiten', u'und', u'der', u'heimische', u'PC', u'durch', u'Internet', u'und', u'Smartphone', u'keine', u'Rolle', u'mehr', u'spielen', u':', u'Fuer', u'die', u'\xab', u'Out', u'of', u'home', u'\xbb', u'-Werbung', u'sind', u'das', u'guenstige', u'Entwicklungen', u'.']
[u'Zudem', u'verbringen', u'sie', u'weniger', u'Zeit', u'zu', u'Hause', u',', u'weil', u'f

In [16]:
from Truecaser import *
import nltk
import string
import PredictTruecaser
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sacremoses import MosesTokenizer, MosesDetokenizer # detokenizing package
md = MosesDetokenizer() 

C:\ProgramData\Anaconda3\envs\py27\lib\site-packages\sacremoses\truecase.py:26: UserWarning: You should really be using Python3!!! Tick tock, tick tock, https://pythonclock.org/
  "You should really be using Python3!!! "


In [17]:
# Define a test sentence for demonstration
test_sentence = 'dieses wort ist falsch geschrieben. dieser satz ist ein test.'

In [18]:
corr_sent = getTrueCase(nltk.word_tokenize(test_sentence), 'as-is', wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, 
                       trigramDist)
corr_sent = md.detokenize(corr_sent).replace(u"``", u' " ').replace(u"''", u' " ').replace(u"'", u" ")
print(corr_sent)

Dieses Wort ist falsch geschrieben. Dieser Satz ist ein Test.


In [26]:
# Load articles that we want to correct
data_truecase = pd.read_csv(os.getcwd().replace('truecaser-master', '') + 'dpa_case_fix.csv', encoding = 'utf-8', sep=';', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'},
                  converters = {'paragraphs': literal_eval})

In [27]:
# An example of an article with incorrect capitalization
data_truecase['texts'].iloc[15]

u"\xabNeue Osnabr\xfccker Zeitung\xbb zu deutsch-sowjetische Beziehungen. es war ein gutes omen fuer die beiderseitigen beziehungen, dass helmut kohl erstmalig im moskauer fernsehen den voelkern der sowjetunion die neujahrsgruesse und -wuensche der deutschen uebermitteln konnte. solche gesten dienen dem verstaendnis untereinander und foerdern die vertrauensbildung. zugleich hat dieser auftritt des kanzlers besonderen symbolwert. er kennzeichnet den tiefgreifenden wandel, der sich im vergangenen jahr im verhaeltnis zur oestlichen grossmacht vollzogen hat. dass der regierungschef eines vereinten deutschland 1991 zu recht von einer ''neu begruendeten freundschaft'' sprechen konnte, waere selbst vor einem jahr kaum vorstellbar gewesen. ein indiz mehr, wie schnell sich das rad der geschichte gedreht hat. dennoch ist der ausblick auf die weitere entwicklung nicht frei von sorgen. der ruecktritt aussenminister schewardnadses, ..., bedeutet einen unsicherheitsfaktor in diesem fuer das verhaelt

In [28]:
# Use the trained truecasing model to correct articles
truecase_art = []
truecase_titles = []

for i, text in data_truecase['texts'].iteritems():
    title = data_truecase.loc[i, 'title']
    
    # Check if the title has at least one uppercase letter, or if it contains 'Ae', 'Ue', 'Oe', or '='
    should_truecase_title = not any(c.isupper() for c in title) or any(substr in title for substr in ['Ae', 'Ue', 'Oe', '='])
    
    # Truecase title if it doesn't have uppercase letters or if it contains 'Ae', 'Ue', 'Oe', or '='
    if should_truecase_title:
        title_truecased = getTrueCase(nltk.word_tokenize(title.lower()), 'as-is', wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)
        title_truecased = md.detokenize(title_truecased).replace(u"``", u' " ').replace(u"''", u' " ').replace(u"'", u" ")
    else:
        title_truecased = title
    
    # Remove the original title from the text
    text_without_title = re.sub(re.escape(title), '', text, count=1)
    
    # Truecase the text without the title
    truecased_text = getTrueCase(nltk.word_tokenize(text_without_title.lower()), 'as-is', wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)
    truecased_text = md.detokenize(truecased_text).replace(u"``", u' " ').replace(u"''", u' " ').replace(u"'", u" ")
    
    # Merge the truecased title with the truecased text
    truecased_article = title_truecased + ' ' + truecased_text
    
    truecase_art.append(truecased_article)
    truecase_titles.append(title_truecased)

In [29]:
truecase_art[15]

u'\xabNeue Osnabr\xfccker Zeitung\xbb zu deutsch-sowjetische Beziehungen. Es war ein gutes Omen fuer die beiderseitigen Beziehungen, dass Helmut Kohl erstmalig im Moskauer Fernsehen den Voelkern der Sowjetunion die Neujahrsgruesse und -wuensche der Deutschen uebermitteln konnte. Solche Gesten dienen dem Verstaendnis untereinander und foerdern die Vertrauensbildung. Zugleich hat dieser Auftritt des Kanzlers besonderen Symbolwert. Er kennzeichnet den tiefgreifenden Wandel, der sich im vergangenen Jahr im Verhaeltnis zur oestlichen Grossmacht vollzogen hat. Dass der Regierungschef eines vereinten Deutschland 1991 zu Recht von einer  " neu begruendeten Freundschaft  " sprechen konnte, waere selbst vor einem Jahr kaum vorstellbar gewesen. Ein Indiz mehr, wie schnell sich das Rad der Geschichte gedreht hat. Dennoch ist der Ausblick auf die weitere Entwicklung nicht frei von Sorgen. Der Ruecktritt Aussenminister Schewardnadses,..., bedeutet einen Unsicherheitsfaktor in diesem fuer das Verhael

In [30]:
data_truecase['texts'] = truecase_art
data_truecase['title'] = truecase_titles

In [31]:
data_truecase['texts'].iloc[15]

u'\xabNeue Osnabr\xfccker Zeitung\xbb zu deutsch-sowjetische Beziehungen. Es war ein gutes Omen fuer die beiderseitigen Beziehungen, dass Helmut Kohl erstmalig im Moskauer Fernsehen den Voelkern der Sowjetunion die Neujahrsgruesse und -wuensche der Deutschen uebermitteln konnte. Solche Gesten dienen dem Verstaendnis untereinander und foerdern die Vertrauensbildung. Zugleich hat dieser Auftritt des Kanzlers besonderen Symbolwert. Er kennzeichnet den tiefgreifenden Wandel, der sich im vergangenen Jahr im Verhaeltnis zur oestlichen Grossmacht vollzogen hat. Dass der Regierungschef eines vereinten Deutschland 1991 zu Recht von einer  " neu begruendeten Freundschaft  " sprechen konnte, waere selbst vor einem Jahr kaum vorstellbar gewesen. Ein Indiz mehr, wie schnell sich das Rad der Geschichte gedreht hat. Dennoch ist der Ausblick auf die weitere Entwicklung nicht frei von Sorgen. Der Ruecktritt Aussenminister Schewardnadses,..., bedeutet einen Unsicherheitsfaktor in diesem fuer das Verhael

In [32]:
data_truecase.to_csv(os.path.join(os.getcwd().replace('truecaser-master', ''), 'dpa_cases_fixed.csv'), encoding='utf-8-sig', sep = ';')